In [1]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()


In [2]:
import pandas as pd

df = pd.read_json('data/1000_posts.json')

entities_result = []
annotations_result = []
counter = 0
for index, row in df.iloc[0:100,:].iterrows():
    counter += 1
    print(counter)
    texts = [row['init_post']] + row['comments']
    for text in texts:
        document = types.Document(
                    content=text,
                    type=enums.Document.Type.PLAIN_TEXT,
                    language='en')
        entities = client.analyze_entity_sentiment(document).entities
        entities_result.append(entities)
        
        annotations = client.analyze_sentiment(document=document)
        annotations_result.append({"score": annotations.document_sentiment.score,
               "magnitude": annotations.document_sentiment.magnitude})

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [3]:
processed_docword_sentiment = {}
for i in range(len(entities_result)):
    for j in range(len(entities_result[i])):
        if(entities_result[i][j].name not in processed_docword_sentiment.keys()):
            processed_docword_sentiment[entities_result[i][j].name] = {'score':[entities_result[i][j].sentiment.score],
                                                             'magnitude':[entities_result[i][j].sentiment.magnitude]}
        else:
            processed_docword_sentiment[entities_result[i][j].name]['score'].append(entities_result[i][j].sentiment.score)
            processed_docword_sentiment[entities_result[i][j].name]['magnitude'].append(entities_result[i][j].sentiment.magnitude)

In [4]:
import json

with open('data/document_entity_sentiment.json', 'w') as outfile:
    json.dump(processed_docword_sentiment, outfile, indent=4, ensure_ascii=False)
    
with open('data/document_sentiment.json', 'w') as outfile:
    json.dump(annotations_result, outfile, indent=4, ensure_ascii=False)

In [15]:
# Detects entities in the document. You can also analyze HTML with:
#   document.type == enums.Document.Type.HTML
entities = client.analyze_entities(document).entities

# entity types from enums.Entity.Type
entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
               'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')

for entity in entities:
    print('=' * 20)
    print(u'{:<16}: {}'.format('name', entity.name))
    print(u'{:<16}: {}'.format('type', entity_type[entity.type]))
    print(u'{:<16}: {}'.format('metadata', entity.metadata))
    print(u'{:<16}: {}'.format('salience', entity.salience))
    print(u'{:<16}: {}'.format('wikipedia_url',
          entity.metadata.get('wikipedia_url', '-')))

name            : Obama
type            : PERSON
metadata        : {'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'mid': '/m/02mjmr'}
salience        : 0.8497163653373718
wikipedia_url   : https://en.wikipedia.org/wiki/Barack_Obama
name            : Trump
type            : PERSON
metadata        : {'wikipedia_url': 'https://en.wikipedia.org/wiki/Donald_Trump', 'mid': '/m/0cqt90'}
salience        : 0.09013491868972778
wikipedia_url   : https://en.wikipedia.org/wiki/Donald_Trump
name            : states
type            : LOCATION
metadata        : {}
salience        : 0.060148686170578
wikipedia_url   : -
